In [49]:
import json
import numpy as np
import pandas as pd
import lxml

# Import SAP data previously mapped for ZA1

15Dec_D365_orders_ZA1.csv

In [50]:
df5 = pd.read_csv("./data/ZA1/15Dec_D365_orders_ZA1.csv", low_memory=False)

# Import Products, Customers and Warehouses for the Legal Entity

In [51]:
LE = input('Enter the legal entity company code')
# NA1, MZ1, UG1

In [52]:
if LE == 'NA1':
    LE_Products = pd.read_csv("./data/NA1/Group1Products_NA1.csv", delimiter="\t")

if LE == 'MZ1':
    LE_Products = pd.read_csv("./data/MZ1/Group1Products_MZ1.csv", delimiter=",")

if LE == 'UG1':
    LE_Products = pd.read_csv("./data/UG1/Group1Products_UG1.csv", delimiter=",")
    

In [53]:
LE_Products

,D365_ItemNo_LE,Material_Description_LE
0,10491,FANTA CASHEW 330ML 4X6C
1,2225,SPAR MORANGO 1L 6 S/W NP
2,4996,MONSTER ULTRA ZERO 06X04 500C
3,6032,SPAR C/SODA 330ML 4X6C
4,10805,CAPPY NEC EXOT 06X04 330ML EXP
...,...,...
107,10382,PREDATOR PURPLE 06X04 500ML CAN
108,2249,SPAR MORANGO 330ML 4X6C
109,3240,FANTA GRAPE 01X06 2000 SHR
110,3841,MAZOE BLACKBERRY 2L 6 PET


In [54]:
if LE == 'NA1':
    LE_Customers = pd.read_csv("./data/NA1/NA1_Customers.csv", delimiter="\t")
    LE_Customers.rename(columns={'Validated Customer Accounts': 'D365_Cust_No_LE'}, inplace=True)
 

if LE == 'MZ1':
    LE_Customers = pd.read_csv("./data/MZ1/PERF01_MZ1_Customers.csv", delimiter="\t")    
    LE_Customers.rename(columns={'CustomerAccount': 'D365_Cust_No_LE'}, inplace=True)
    
if LE == 'UG1':
    LE_Customers = pd.read_csv("./data/UG1/IPT3 UG1- Customers.csv", delimiter=",")    
    LE_Customers.rename(columns={'CustomerAccount': 'D365_Cust_No_LE'}, inplace=True)
    LE_Customers = LE_Customers[['D365_Cust_No_LE']].copy()
    # Name


In [55]:
# Read list of direct customers from CE, and merge that with the F&O customers per legal entity

CE_direct_customers = pd.read_csv('./data/Group1Customers.csv', delimiter=";")
CE_direct_customers.drop(columns={'name', 'ccba_cicorderplacementrulename'}, inplace=True, axis=1)
CE_direct_customers.rename(columns={'accountnumber': 'D365_Cust_No_LE'}, inplace=True)
CE_direct_customers = CE_direct_customers[CE_direct_customers['msdyn_companyname'] == LE].copy()
CE_direct_customers.drop(columns={'msdyn_companyname'}, inplace=True, axis=1)

In [61]:
# Merge based on Index
LE_Customers = pd.merge(
    LE_Customers,
    CE_direct_customers,
    how="inner",
    on="D365_Cust_No_LE",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [446]:
if LE == 'NA1':
    LE_Warehouses = pd.read_csv("./data/NA1/NA1_Validated  IPT3 Warehouse Export  NA1 Warehouses.csv", delimiter=",")
    LE_Warehouses.rename(columns={'D365_Del_Location': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365_Del_Location

if LE == 'MZ1':
    LE_Warehouses = pd.read_csv("./data/MZ1/IPT3 MZ1 PERF01 Warehouse.csv", delimiter=",")
    LE_Warehouses.rename(columns={'D365DELIVERYLOCATION': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365DELIVERYLOCATION

if LE == 'UG1':
    LE_Warehouses = pd.read_csv("./data/UG1/IPT3 WMS UG1-Warehouses.csv", delimiter=",")    
    LE_Warehouses.rename(columns={'D365_Del_Loc': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365_Del_Loc

In [447]:
LE_Warehouses

,D365_Del_Loc_LE
0,MZ1-MZ010B
1,MZ1-MZ010Q
2,MZ1-MZ011B
3,MZ1-MZ011Q
4,MZ1-MZ012B
5,MZ1-MZ012Q
6,MZ1-MZ013B


## Get the top n warehouses and filter the dataframe for those

In [448]:
# Group by site_id and count unique Order Numbers
#warehouse_order_counts = df5.groupby('site_id')['Order Number'].nunique().reset_index(name='OrderLinesCount')
warehouse_order_counts = df5.groupby('D365_Del_Loc')['Order Number'].nunique().reset_index(name='OrderLinesCount')
#D365_Del_Loc 

# Sort by OrderLinesCount in descending order and get the top 9 warehouses
#top_warehouses = warehouse_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Warehouses))
top_warehouses = warehouse_order_counts.sort_values(by='OrderLinesCount', ascending=False)

#top_warehouses['Grouping'] = (top_warehouses.index % 7) + 1


In [449]:
top_warehouses.reset_index(inplace=True, drop=True)
top_warehouses['Grouping'] = (top_warehouses.index % len(LE_Warehouses))

In [450]:
top_warehouses

,D365_Del_Loc,OrderLinesCount,Grouping
0,ZA1-ZA014B,1113,0
1,ZA1-ZA016G,983,1
2,ZA1-ZA516B,951,2
3,ZA1-ZA020B,922,3
4,ZA1-ZA032B,921,4
...,...,...,...
82,ZA1-ZA034B,16,5
83,ZA1-ZA200B,13,6
84,ZA1-ZA565B,10,0
85,ZA1-ZA039B,5,1


In [451]:
# Filter the original DataFrame to only include rows where site_id is in the list of top warehouses
#top_warehouse_ids = top_warehouses['D365_Del_Loc'].tolist()
#df5_W = df5[df5['D365_Del_Loc'].isin(top_warehouse_ids)]
df5_W = df5.copy()

In [452]:
top_warehouses.reset_index(inplace=True, drop=True)

## Get the top n customers and filter the dataframe for those

In [453]:
# Group by D365_Cust_No and count unique Order Numbers
customer_order_counts = df5_W.groupby('D365_Cust_No')['Order Number'].nunique().reset_index(name='OrderLinesCount')

# Sort by OrderLinesCount in descending order and get the top n customers
#top_customers = customer_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Customers))
top_customers = customer_order_counts.sort_values(by='OrderLinesCount', ascending=False)

In [454]:
top_customers.reset_index(inplace=True, drop=True)

In [455]:
top_customers.reset_index(inplace=True, drop=True)
top_customers['Grouping'] = (top_customers.index % len(LE_Customers))

In [456]:
top_customers

,D365_Cust_No,OrderLinesCount,Grouping
0,10017885,73,0
1,10004521,63,1
2,10017888,57,2
3,10004547,56,3
4,10017883,47,4
...,...,...,...
7017,10021202,1,47
7018,10053913,1,48
7019,10054011,1,49
7020,10019883,1,50


In [457]:
#top_customer_ids = top_customers['D365_Cust_No'].tolist()
#df5_W_C = df5_W[df5_W['D365_Cust_No'].isin(top_customer_ids)]
df5_W_C = df5_W.copy()

## Get the top n products and filter the dataframe for those

In [458]:
product_order_counts = df5_W_C.groupby('D365_ItemNo')['Order Number'].nunique().reset_index(name='OrderLinesCount')


top_products = product_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Products))


print(top_products)

    D365_ItemNo  OrderLinesCount
126        1040            12704
310        7579            10761
267        6540            10031
100       10207             9863
227        5740             9383
..          ...              ...
331        7889             1627
326        7757             1622
170        2252             1497
208        4995             1488
209       4995M             1488

[112 rows x 2 columns]


In [459]:
top_product_ids = top_products['D365_ItemNo'].tolist()
df5_W_C_P = df5_W_C[df5_W_C['D365_ItemNo'].isin(top_product_ids)]

In [460]:
top_products.reset_index(inplace=True, drop=True)

## Now replace warehouses, customers and products for the data in the new legal entity
Extend the dataframes produced above, top_warehouses, top_customers and top_products by adding a column (for the same number of rows).  The new column contains the corresponding data for the new legal entity.

### Create Products lookup table

In [461]:
# Create a new column 'Index' in both dataframes with current index values
top_products['Index'] = top_products.index
LE_Products['Index'] = LE_Products.index

In [462]:
if len(LE_Products) != len(top_products):
    raise SystemExit("Mismatch in number of products")

In [463]:
LE_Products

,D365_ItemNo_LE,Material_Description_LE,Index
0,10491,FANTA CASHEW 330ML 4X6C,0
1,2225,SPAR MORANGO 1L 6 S/W NP,1
2,4996,MONSTER ULTRA ZERO 06X04 500C,2
3,6032,SPAR C/SODA 330ML 4X6C,3
4,10805,CAPPY NEC EXOT 06X04 330ML EXP,4
...,...,...,...
107,10382,PREDATOR PURPLE 06X04 500ML CAN,107
108,2249,SPAR MORANGO 330ML 4X6C,108
109,3240,FANTA GRAPE 01X06 2000 SHR,109
110,3841,MAZOE BLACKBERRY 2L 6 PET,110


In [464]:
# Merge based on Index
top_products = pd.merge(
    top_products,
    LE_Products,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [465]:
top_products.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

In [466]:
top_products

,D365_ItemNo,D365_ItemNo_LE,Material_Description_LE
0,1040,10491,FANTA CASHEW 330ML 4X6C
1,7579,2225,SPAR MORANGO 1L 6 S/W NP
2,6540,4996,MONSTER ULTRA ZERO 06X04 500C
3,10207,6032,SPAR C/SODA 330ML 4X6C
4,5740,10805,CAPPY NEC EXOT 06X04 330ML EXP
...,...,...,...
107,7889,10382,PREDATOR PURPLE 06X04 500ML CAN
108,7757,2249,SPAR MORANGO 330ML 4X6C
109,2252,3240,FANTA GRAPE 01X06 2000 SHR
110,4995,3841,MAZOE BLACKBERRY 2L 6 PET


### Create Customers lookup table

In [467]:
# Create a new column 'Index' in both dataframes with current index values
#top_customers['Index'] = top_customers.index
LE_Customers['Index'] = LE_Customers.index

In [468]:
top_customers.rename(columns={'Grouping': 'Index'}, inplace=True)

In [469]:
#if len(LE_Customers) != len(top_customers):
#    raise SystemExit("Mismatch in number of customers")

In [470]:
#LE_Customers = LE_Customers.drop_duplicates().copy()

In [471]:
# Merge based on Index
top_customers = pd.merge(
    top_customers,
    LE_Customers,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [472]:
top_customers.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

### Create Warehouses lookup table

In [473]:
# Create a new column 'Index' in both dataframes with current index values
#top_warehouses['Index'] = top_warehouses.index
LE_Warehouses['Index'] = LE_Warehouses.index

In [474]:
top_warehouses.rename(columns={'Grouping': 'Index'}, inplace=True)

In [475]:
#if len(LE_Warehouses) != len(top_warehouses):
#    raise SystemExit("Mismatch in number of warehouses")

In [476]:
# Merge based on Index
top_warehouses = pd.merge(
    top_warehouses,
    LE_Warehouses,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [477]:
top_warehouses.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

In [478]:
top_warehouses

,D365_Del_Loc,D365_Del_Loc_LE
0,ZA1-ZA014B,MZ1-MZ010B
1,ZA1-ZA550B,MZ1-MZ010B
2,ZA1-ZA505B,MZ1-MZ010B
3,ZA1-ZA024F,MZ1-MZ010B
4,ZA1-ZA525B,MZ1-MZ010B
...,...,...
82,ZA1-ZA052B,MZ1-MZ013B
83,ZA1-ZA544B,MZ1-MZ013B
84,ZA1-ZA511B,MZ1-MZ013B
85,ZA1-ZA527B,MZ1-MZ013B


### Merge Warehouses lookup table with SAP data

In [479]:
df5_1 = pd.merge(
    df5_W_C_P,
    top_warehouses,
    how="left",
    on="D365_Del_Loc",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [480]:
df5_W_C_P.info()

<class 'pandas.core.frame.DataFrame'>
Index: 726285 entries, 0 to 871830
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              726285 non-null  object
 1   Order Number              726285 non-null  int64 
 2   Material_Description      726285 non-null  object
 3   BaseDate                  726285 non-null  object
 4   BaseHour                  726285 non-null  int64 
 5   Cases                     726285 non-null  int64 
 6   Bill I                    722983 non-null  object
 7   Source Channel            726285 non-null  object
 8   Order Category            726285 non-null  object
 9   D365_ItemNo               726285 non-null  object
 10  Material No               726285 non-null  object
 11  MOD                       726285 non-null  int64 
 12  site_id                   726285 non-null  object
 13  WAREHOUSELOCATIONID       726285 non-null  object
 14  D365_Del_

In [481]:
df5_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726285 entries, 0 to 726284
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              726285 non-null  object
 1   Order Number              726285 non-null  int64 
 2   Material_Description      726285 non-null  object
 3   BaseDate                  726285 non-null  object
 4   BaseHour                  726285 non-null  int64 
 5   Cases                     726285 non-null  int64 
 6   Bill I                    722983 non-null  object
 7   Source Channel            726285 non-null  object
 8   Order Category            726285 non-null  object
 9   D365_ItemNo               726285 non-null  object
 10  Material No               726285 non-null  object
 11  MOD                       726285 non-null  int64 
 12  site_id                   726285 non-null  object
 13  WAREHOUSELOCATIONID       726285 non-null  object
 14  D365

### Merge Products lookup table with SAP data

In [482]:
df5_2 = pd.merge(
    df5_1,
    top_products,
    how="left",
    on="D365_ItemNo",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [483]:
df5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726285 entries, 0 to 726284
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              726285 non-null  object
 1   Order Number              726285 non-null  int64 
 2   Material_Description      726285 non-null  object
 3   BaseDate                  726285 non-null  object
 4   BaseHour                  726285 non-null  int64 
 5   Cases                     726285 non-null  int64 
 6   Bill I                    722983 non-null  object
 7   Source Channel            726285 non-null  object
 8   Order Category            726285 non-null  object
 9   D365_ItemNo               726285 non-null  object
 10  Material No               726285 non-null  object
 11  MOD                       726285 non-null  int64 
 12  site_id                   726285 non-null  object
 13  WAREHOUSELOCATIONID       726285 non-null  object
 14  D365

### Merge Customers lookup table with SAP data

In [484]:
df5_3 = pd.merge(
    df5_2,
    top_customers,
    how="left",
    on="D365_Cust_No",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [485]:
df5_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726285 entries, 0 to 726284
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              726285 non-null  object
 1   Order Number              726285 non-null  int64 
 2   Material_Description      726285 non-null  object
 3   BaseDate                  726285 non-null  object
 4   BaseHour                  726285 non-null  int64 
 5   Cases                     726285 non-null  int64 
 6   Bill I                    722983 non-null  object
 7   Source Channel            726285 non-null  object
 8   Order Category            726285 non-null  object
 9   D365_ItemNo               726285 non-null  object
 10  Material No               726285 non-null  object
 11  MOD                       726285 non-null  int64 
 12  site_id                   726285 non-null  object
 13  WAREHOUSELOCATIONID       726285 non-null  object
 14  D365

In [486]:
df5_3.drop(columns={'site_id', 'D365_Cust_No', 'Material_Description', 'D365_ItemNo', 'D365_Del_Loc'}, inplace=True, axis=1)

In [487]:
df5_3.rename(columns={'site_id_LE': 'site_id', 'D365_Del_Loc_LE': 'D365_Del_Loc', 'D365_Cust_No_LE': 'D365_Cust_No', 'Material_Description_LE': 'Material_Description', 'D365_ItemNo_LE': 'D365_ItemNo'}, inplace=True)

In [488]:
df5_3['BaseDate'] = pd.to_datetime(df5_3['BaseDate'])

In [489]:
# Drop duplicates based on Customer, D365_ItemNo, and BaseDate.
#df5_3 = df5_3.sort_values('Order Number').drop_duplicates(subset=['D365_Cust_No', 'D365_ItemNo', 'BaseDate'])
df5_3 = df5_3.sort_values('Order Number').drop_duplicates(subset=['Order Number', 'D365_ItemNo', 'BaseDate'])
#df5_3 = df5_3.sort_values('Order Number').drop_duplicates(subset=['ActivityType', 'D365_Cust_No', 'D365_ItemNo', 'BaseDate'])

In [490]:
path = './data/'+ LE + '/15Dec_D365_orders_' + LE + '.feather'
df5_3.to_feather(path)